In [1]:
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("deceptive-opinion.csv")
df.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [3]:
df['deceptive'] = df['deceptive'].map({'truthful': 1, 'deceptive': 0})
df.head()

,deceptive,hotel,polarity,source,text
0,1,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,1,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,1,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,1,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,1,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [4]:
#text_process function for making tokens of data
def text_process(mess):
    return mess.split()

In [5]:
#creating a bow transformer with token analyzer
from sklearn.feature_extraction.text import CountVectorizer


bow_transformer = CountVectorizer(analyzer=text_process).fit(df['text'])

print(len(bow_transformer.vocabulary_))

19294


In [6]:
print(bow_transformer.get_feature_names())

['!', '!!', '!!!', '!!!.', '#', '#1', '#1605', '#1605.', '#2', '#607', "$'s.", '$)', '$*...I', '$1,000', '$1.00.', '$1.25/15mins', '$1.99.', '$10', '$10,000', '$10.', '$10.50/glass', '$10/day', '$100', '$100+', '$100.', '$100.00', '$103.63!!!', '$104/', '$105', '$11', '$110', '$116', '$12', '$125,', '$125/night.', '$129', '$129.', '$129.00', '$13', '$13.00,', '$130/night,', '$139', '$14', '$14!', '$14.00', '$14.95/day', '$149.', '$15', '$15.00', '$15/day', '$150/night', '$1500', '$159', '$16', '$165', '$17', '$170/night', '$175,', '$18', '$180', '$180/nt.', '$19.99', '$191', '$2', '$2,127.00!', '$2.', '$2.00', '$2.00,', '$2.99', '$2/minute.', '$20', '$20)', '$20.', '$20.00', '$20/night', '$200', "$200's.", '$200).', '$200.00', '$200/night', '$200/person', '$203.', '$21', '$21.44', '$214/night', '$219', '$22', '$22/person.', '$220/night', '$24', '$24/days', '$240', '$240.43', '$245', '$249', '$249/night', '$25', '$25.', '$25.00', '$250', '$250/night', '$259', '$26', '$260', '$279', '$28

In [7]:
#print(len(bow_transformer.vocabulary_))
#print(bow_transformer)
#message_bow.shape
#non nulls
#message_bow.nnz
#message_bow

In [8]:
#transforming the text in df to vectors after tokenization using count vectorizer
message_bow=bow_transformer.transform(df['text'])

In [9]:
from sklearn.model_selection import train_test_split
text_train,text_test,label_train,label_test= train_test_split(df['text'],df['deceptive'],test_size=0.3)


In [10]:
#data pipeline is used to store pipeline of workflow
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier ,export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


#creating a pipeline for svm and doing predictions
pipeline_svm=Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('classifier',SVC())
])
pipeline_svm.fit(text_train,label_train)
predictions_svm=pipeline_svm.predict(text_test)
score_svm=cross_val_score(SVC(),message_bow,df['deceptive'],cv=5)

print("Classification report without POS Tagging for 1600 reviews using SVM Classifier")
print(" ")
print("svm",classification_report(label_test,predictions_svm))
print("5 Fold Cross Validation Score:",score_svm)
print("")

print("------------------------------------------------------------------------------")


#creating a pipeline for text processing and fitting our data in the pipeline NB
pipeline=Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('classifier',MultinomialNB())
])

pipeline.fit(text_train,label_train)

#prediction
predictions=pipeline.predict(text_test)
score_NB=cross_val_score(MultinomialNB(),message_bow,df['deceptive'],cv=5)
print("Classification report without POS Tagging for 1600 reviews using MultinomialNB Classifier")
print(" ")
print(classification_report(label_test,predictions))
print("5 Fold Cross Validation Score:",score_NB)
print("")

print("------------------------------------------------------------------------------")

#creating a pipeline for Logistic Regression and doing predictions
pipeline_LR=Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('classifier',LogisticRegression(max_iter=200))
])
pipeline_LR.fit(text_train,label_train)
predictions_LR=pipeline_LR.predict(text_test)
score_LR=cross_val_score(LogisticRegression(max_iter=200),message_bow,df['deceptive'],cv=5)
print("Classification report without POS Tagging for 1600 reviews using Logistic Regression Classifier")
print(" ")
print(classification_report(label_test,predictions_LR))
print("5 Fold Cross Validation Score:",score_LR)
print("")

print("------------------------------------------------------------------------------")

#creating a pipeline for DecisionTreeClassifier and doing predictions
pipeline_DT=Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('classifier',DecisionTreeClassifier())
])
pipeline_DT.fit(text_train,label_train)
predictions_DT=pipeline_DT.predict(text_test)
score_DT=cross_val_score(DecisionTreeClassifier(),message_bow,df['deceptive'],cv=5)
print("Classification report without POS Tagging for 1600 reviews using Decision Tree Classifier")
print(" ")
print(classification_report(label_test,predictions_DT))
print("5 Fold Cross Validation Score:",score_DT)
print("")

print("------------------------------------------------------------------------------")

#creating a pipeline for RandomForestClassifier and doing predictions
pipeline_RF=Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('classifier',RandomForestClassifier())
])
pipeline_RF.fit(text_train,label_train)
predictions_RF=pipeline_RF.predict(text_test)
score_RF=cross_val_score(RandomForestClassifier(),message_bow,df['deceptive'],cv=5)
print("Classification report without POS Tagging for 1600 reviews using Random Forest Classifier")
print(" ")
print(classification_report(label_test,predictions_RF))
print("5 Fold Cross Validation Score:",score_RF)
print("")

print("------------------------------------------------------------------------------")


Classification report without POS Tagging for 1600 reviews using SVM Classifier
 
svm               precision    recall  f1-score   support

           0       0.80      0.27      0.41       239
           1       0.56      0.93      0.70       241

    accuracy                           0.60       480
   macro avg       0.68      0.60      0.55       480
weighted avg       0.68      0.60      0.56       480

5 Fold Cross Validation Score: [0.5625   0.546875 0.65     0.61875  0.628125]

------------------------------------------------------------------------------
Classification report without POS Tagging for 1600 reviews using MultinomialNB Classifier
 
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       239
           1       0.92      0.77      0.84       241

    accuracy                           0.85       480
   macro avg       0.86      0.85      0.85       480
weighted avg       0.86      0.85      0.85       480

5 Fold Cro